### Introduction 

When can either:
1. emphazise declared features in loss criterion

By emphazising weights to MSE we can emphazise more on certain features.
This may help to take individual requirements more into account e.g. a coach wants a player like Rodri especially like certain attributes.
In this case chapter we emphazise on 'Receiving_Succ', 'Shots'.

2. add more importance by multiplying the inpute features by some weights

### Load data

In [5]:
import pandas as pd 
import utils

# Set seed for reproducibility
SEED = 42

df, player_info = utils.load_player_statistics()

In [6]:
player_info_cols = ['Season', 'League', 'Team', 'Player', 'Nation', 'Pos','Global Pos', 'Age','Matches Played','Playing Time_Starts','Playing Time_Min','Playing Time_90s']

def_cols = ['Tackles_Att', 'Tackles_Def 3rd', 'Tackles_Mid 3rd',
        'Tackles_Att 3rd','Interceptions', 'Clearances','Errors',
        'Dribblers_Tkl_Succ','Dribblers_Tkl_Att','Dribblers_Tkl_Lost',
        'Blocks_Total', 'Blocks_Shots',
        ]

possession_cols = ['Touches_Number', 'Touches_Def Pen', 'Touches_Def 3rd',
       'Touches_Mid 3rd', 'Touches_Att 3rd', 'Touches_Att Pen','Take-Ons_Att', 'Take-Ons_Succ',
       'Take-Ons_Tkld','Take-Ons_Tkld%','Carries_Number', 'Carries_TotDist',
       'Carries_PrgDist', 'Carries_PrgC', 'Carries_1/3', 'Carries_CPA',
       'Carries_Mis', 'Carries_Dis','Receiving_Succ', 'Receiving_PrgR'
        ]
passing_cols = ['Total_Cmp', 'Total_Att', 'Total_Cmp%', 'Total_TotDist',
       'Total_PrgDist','Short_Cmp', 'Short_Att', 'Short_Cmp%', 'Medium_Cmp',
       'Medium_Att', 'Medium_Cmp%', 'Long_Cmp', 'Long_Att', 'Long_Cmp%','Assists', 'Key Passes', 'Passes_to_1/3',
       'Passes_to_Penalt_Area', 'Crosses_into_Penalty_Area','Progressive Passes']

gsc_cols = ['SCA', 'SCA90', 'SCA Types_PassLive', 'SCA Types_PassDead',
       'SCA Types_TO', 'SCA Types_Sh', 'SCA Types_Fld', 'SCA Types_Def', 'GCA',
       'GCA90', 'GCA Types_PassLive', 'GCA Types_PassDead', 'GCA Types_TO',
       'GCA Types_Sh', 'GCA Types_Fld', 'GCA Types_Def'
        ]

shooting_cols = ['Goals', 'Shots', 'SoT', 'SoT%', 'Shots/90','Goals/Shot', 'Goals/SoT']

adv_goalkeeping = ['Goals_GA', 'Goals_PKA', 'Goals_FK', 'Goals_CK', 'Goals_OG', 'PSxG',
       'PSxG/SoT', 'PSxG+/-', '/90', 'Launched_Cmp', 'Launched_Att',
       'Launched_Cmp%', 'Passes_Att', 'Passes_Thr', 'Passes_Launch%',
       'Passes_AvgLen', 'Goal Kicks_Att', 'Goal Kicks_Launch%',
       'Goal Kicks_AvgLen', 'Opp', 'Stp', 'Stp%', '#OPA', '#OPA/90',
       'AvgDist']
goalkeeping_cols = ['GA',
       'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%',
       'Penalty Kicks_PKatt', 'Penalty Kicks_PKA', 'Penalty Kicks_PKsv',
       'Penalty Kicks_PKm', 'Penalty Kicks_Save%']


misc_cols = ['2CrdY', 'Fls', 'Fld', 'Off', 'Crs','PKwon', 
            'PKcon', 'OG', 'Recov', 'Aerial Duels_Won',
            'Aerial Duels_Lost'
        ]



playing_time_cols = ['Playing Time_Minutes', 'Playing Time_Mn/MP','Starts', 'Mn/Start', 'Compl',
                     'Subs', 'unSub', 'PPM','onG', 'onGA','On-Off'
                    ]

feature_vector = (player_info_cols + def_cols + possession_cols + passing_cols + 
               gsc_cols + shooting_cols + adv_goalkeeping + goalkeeping_cols + 
               misc_cols + playing_time_cols)

### 1. Emphazise in loss criterion

#### set up dataloader

In [7]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, Normalizer

# scale data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# set seeds
torch.manual_seed(SEED)
np.random.seed(SEED)

# Convert to PyTorch tensors
data_tensor = torch.tensor(df_scaled, dtype=torch.float32)

# Create a DataLoader
batch_size = 32
dataset = TensorDataset(data_tensor, data_tensor)  # Input and target are the same for autoencoder
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### define criterion

#### delcare features

In [8]:
# focus on carry dimension
weight_dimension= ['Carries_TotDist', 'Carries_PrgDist', 'Carries_PrgC', 'Carries_1/3']

start_idx = df.columns.get_loc(weight_dimension[0])
end_idx = df.columns.get_loc(weight_dimension[-1])
start_idx, end_idx

def add_weights_to_feature(start_idx, end_idx, df):
    feature_weights = [1] * df.shape[1]

    for i in range(start_idx, end_idx + 1):
        feature_weights[i] =  2.0
    
    return feature_weights

features_weights = add_weights_to_feature(start_idx,end_idx,df)

#### weighted criterion

In [9]:
import torch.nn as nn

class WeightedMSELoss(nn.Module):
    def __init__(self, feature_weights):
        super(WeightedMSELoss, self).__init__()
        self.feature_weights = torch.tensor(feature_weights, dtype=torch.float32)
    
    def forward(self, output, target):
        # Compute the MSE loss
        mse_loss = nn.MSELoss(reduction='none')(output, target)
        # Apply the feature weights
        weighted_loss = mse_loss * self.feature_weights
        # Return the mean of the weighted losses
        return weighted_loss.mean()

# Instantiate the custom loss function
weighted_criterion = WeightedMSELoss(features_weights)

### create model

In [10]:
import autoencoder
import torch.optim as optim

# Define model parameters
input_dim = df_scaled.shape[1]  # Input dimensions = number of features
encoding_dim = 50  # Size of the bottleneck layer

# Create model
model = autoencoder.Autoencoder(input_dim, encoding_dim)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Training

In [11]:
model = autoencoder.train_model(
    model=model,
    dataloader=dataloader,
    criterion=weighted_criterion,
    optimizer=optimizer,
    num_epochs=50
)

Epoch [10/50], Loss: 1.9777
Epoch [20/50], Loss: 1.8371
Epoch [30/50], Loss: 0.4829
Epoch [40/50], Loss: 0.3520
Epoch [50/50], Loss: 0.2775


#### Bottleneck Layer - latent vector - embeddings

In [12]:
# Pushing whole data set through network to get latent vector
embeddings = model.get_embeddings(data_tensor)

#### Top k Similar Players

In [13]:
top_k_similar_players = utils.get_top_k_similar_players(
    embeddings=embeddings,
    query_index=795,
    player_info=player_info,
    top_k=10
)

top_k_similar_players

,Season,League,Team,Player,Nation,Pos,Global Pos,Age,Matches Played,Playing Time_Starts,Playing Time_Min,Playing Time_90s,cosine_similarity
Player,,,,,,,,,,,,,
Christopher Nkunku,2022-2023,Bundesliga,RB Leipzig,Christopher Nkunku,FRA,"FW,MF",FW,24.0,25.0,20.0,1897.0,21.1,1.000000
Alexis Sánchez,2022-2023,Ligue1,Marseille,Alexis Sánchez,CHI,"FW,MF",FW,33.0,35.0,32.0,2679.0,29.8,0.968053
Folarin Balogun,2022-2023,Ligue1,Reims,Folarin Balogun,USA,FW,FW,21.0,37.0,34.0,2999.0,33.3,0.963234
Rafa Silva,2022-2023,PrimeiraLiga,Benfica,Rafa Silva,POR,"MF,FW",MF,29.0,28.0,26.0,2174.0,24.2,0.963097
Ciro Immobile,2022-2023,SeriaA,Lazio,Ciro Immobile,ITA,FW,FW,32.0,31.0,27.0,2219.0,24.7,0.960291
Ademola Lookman,2022-2023,SeriaA,Atalanta,Ademola Lookman,NGA,"FW,MF",FW,24.0,31.0,20.0,1729.0,19.2,0.957202
Elye Wahi,2022-2023,Ligue1,Montpellier,Elye Wahi,FRA,FW,FW,19.0,33.0,29.0,2513.0,27.9,0.954836
Tammy Abraham,2022-2023,SeriaA,Roma,Tammy Abraham,ENG,FW,FW,24.0,38.0,24.0,2189.0,24.3,0.952587
Breel Embolo,2022-2023,Ligue1,Monaco,Breel Embolo,SUI,FW,FW,25.0,32.0,19.0,1859.0,20.7,0.952140
